# Scrapping IMDB top 250 (https://www.imdb.com/search/title/?groups=top_250) and leading webpages

### Items Scrapped
        • Movie Name
        • Release Year
        • Duaration
        • IMDB Rating
        • Posters
        • Trailers
        • Photos
        • Genre(s)
        • Description
        • Director
        • Writer
        • Cast
        
        

Importing modules


In [38]:
from selenium import webdriver
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By
import time
import requests
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
#we will use random to specify sleep time in between the process.
import random

Open Driver 

In [39]:
URL= "https://www.imdb.com/search/title/?groups=top_250"
driver = webdriver.Chrome()
driver.get(URL)
time.sleep(0.33)

First load entire HTML content of the page by clicking the show more button 5 times.

In [40]:
from selenium.common.exceptions import TimeoutException

driver.maximize_window()
time.sleep(2) 

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(2)
for i in range(0, 4):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.random())
    
    try:
        next_page_button = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'button.ipc-see-more__button[role="button"]'))
        )
        body = driver.find_element(By.CSS_SELECTOR, 'body')
        for k in range(0, 8):
            body.send_keys(Keys.ARROW_UP)
            time.sleep(random.random())   
        next_page_button.click()
        time.sleep(random.randint(3, 5))
    except TimeoutException:
        print("TimeoutException: No more pages to load.")
        break
    

 

TimeoutException: No more pages to load.


Now take out following information out of the rendered webpage:
    
    Links to individual webpages
    Names of movies
    Rating(IMDB)
    Duration of movie
    Movie Poster
    Moive summary 
    

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Initialize the WebDriver session
driver = webdriver.Chrome()

try:
    # Open the desired webpage
    driver.get('http://example.com')

    links=[]
    movie_names=[]
    ratings=[]
    year=[]
    duration=[]
    poster=[]
    summary=[]
    time.sleep(1)

    # Ensure the session is active and find the elements
    list_of_titles = driver.find_elements(By.CSS_SELECTOR, 'h3.ipc-title__text')
    list_of_links = driver.find_elements(By.CSS_SELECTOR,'a.ipc-title-link-wrapper[tabindex="0"]')
    list_of_summary = driver.find_elements(By.CSS_SELECTOR,'div.ipc-html-content-inner-div')
    list_of_ratings = driver.find_elements(By.CSS_SELECTOR,'span.ipc-rating-star--imdb[data-testid="ratingGroup--imdb-rating"]')
    raw_data = driver.find_elements(By.CSS_SELECTOR,'span.dli-title-metadata-item')
    images = driver.find_elements(By.CSS_SELECTOR,'img.ipc-image[loading="lazy"]')

    # Process the list_of_titles as needed
    for title in list_of_titles:
        print(title.text)

finally:
    # Close the WebDriver session
    driver.quit()


In [42]:
#now taking out all the necessary info from the raw data extracted using the find_elements method

# Ensure the lists have at least 250 elements
min_length = min(len(list_of_titles), len(list_of_links), len(list_of_summary), len(list_of_ratings), len(images))
num_elements = min(250, min_length)

for i in range(num_elements):
    
    m = list_of_titles[i].text
    replaced_m = (re.sub("^(\d)*\. ",'',m))
    final = re.sub("Recently viewed","",replaced_m)
    movie_names.append(final)
    
    links.append(list_of_links[i].get_attribute("href"))

    summary.append(list_of_summary[i].text)

    ratings.append(list_of_ratings[i].text[:3])

    poster.append(images[i].get_attribute('src'))

#Scraping release year and duration
corrected_data=[]
for i in range(len(raw_data)):
    corrected_data.append(raw_data[i].text)
corrected_data.insert(83,"R")
corrected_data.insert(359,"R")
corrected_data.insert(590,"R")
for i in range(len(corrected_data)):
    if (i%3==0):
        year.append(corrected_data[i])
    if (i%3==1):
        duration.append(corrected_data[i])
    


<>:10: SyntaxWarning: invalid escape sequence '\d'
<>:10: SyntaxWarning: invalid escape sequence '\d'
C:\Users\hp\AppData\Local\Temp\ipykernel_17076\3183518457.py:10: SyntaxWarning: invalid escape sequence '\d'
  replaced_m = (re.sub("^(\d)*\. ",'',m))


In [43]:
driver.quit() #it is important to close the driver

### Extracting info out of individual webpages using Beautiful Soup
* Reviews(Critic only)
* Trailer
* Genre(s)
* Cast
* Director 
* Writer
* Photos


>Here I am using BS over find_elements method of selenium because BS supports multi-valued attributes.
This makes searching the tree structure a lot more convenient.


In [44]:
movie_genre=[]
movie_directors=[]
movie_writers=[]
movie_cast=[]
movie_lang=[]
metascores=[]

for URL in links:
    
    response = requests.get(
    url='https://proxy.scrapeops.io/v1/',
    params={
        'api_key': 'f208ed04-171b-49a8-8f53-7c9d22f2c24f',
        'url': URL, 
    },
    )

    soup = BeautifulSoup(response.text,features='html.parser')
    
    #initializing arrays
    genre=[]
    directors=[]
    writers=[]
    cast=[]
    languages=[]
    
    #extracting genre
    try:
        genre_sup_div = soup.find('div',class_="sc-4e4cc5f9-10 jXLkbv")
        genre_div = genre_sup_div.find('div',class_="ipc-chip-list__scroller")
        genre_spans = genre_div.find_all('span')
        for i in genre_spans:
            genre.append(i.text)
    except:
        genre=["Movie"] #There were no genre specified for one movie
    
    #extracting directors
    try:
        dir_ul = soup.find('ul',class_='ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt',attrs={'role':'presentation'})
        dir_lis = dir_ul.find_all('li')
        for i in dir_lis:
            directors.append(i.text)
    except:
        pass #error will not occur, but still for the sake of doing it
    
    #extractinhg writers
    try:
        
        big_ul = soup.find('ul',class_='ipc-metadata-list ipc-metadata-list--dividers-all sc-bfec09a1-8 bHYmJY ipc-metadata-list--base')
        #here writer_uls also includes the ul of director. In the tree structure, the second ul is of writer.
        writer_uls = big_ul.find_all('ul',class_='ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content base')
        writer_lis = writer_uls[1].find_all('li')   
        for i in writer_lis:
            writers.append(i.a.text)   
    except:
        pass#error will not occur, but still for the sake of doing it

    #extracting cast
    try:
        cast_div = soup.find_all('div',class_="sc-bfec09a1-5 hNfYaW",attrs={'data-testid':"title-cast-item"})
        for i in cast_div:
            a_tag_value = i.find('a',class_='sc-bfec09a1-1 gCQkeh').text
            cast.append(a_tag_value)
    except:
        #error will not occur, but still for the sake of doing it
        pass
    #metascore
    try:
        metascore = soup.find('span',class_='metacritic-score-box').text
    except:
        metascore = "N.A."
    #now append to the greater list
    movie_genre.append(genre)
    movie_directors.append(directors)
    movie_writers.append(writers)
    movie_cast.append(cast)
    movie_lang.append(languages)

    if (metascore!=None):
        metascores.append(metascore)
    else:
        #there are some movies for which meta score is not available
        metascores.append("Has Not Been Calculated As Of Now")

### Storing in JSON data format


In [45]:
import json
from textwrap import indent
from matplotlib.font_manager import json_dump

In [46]:
#here movie date is a list of dictionaries. We do so becuase there is a method to convert list of dictionaries to list of json objects.
movie_data=[]

#make a dictionary
for i in range(len(movie_names)):
    movie = {
        "index" : i,
        "name" : movie_names[i],
        "duration" : duration[i],
        "rating" : ratings[i],
        "release_year" : year[i],
        "poster" : poster[i],
        "summary" : summary[i],
        "genre" : movie_genre[i],
        "directors" : movie_directors[i],
        "writers" : movie_writers[i],
        "languages" : movie_lang[i],
        "cast" : movie_cast[i],
        "metascore" : metascores[i],
        "imdb_page" : links[i]
    }
    movie_data.append(movie)

#convert dictionary to jason
json_data = json.dumps(movie_data,indent=4)

with open("movies.json", "w") as outfile:
  outfile.write(json_data)
